# Iterative Prisoner's Dilemma


### Description

The [Prisoner's Dilemma](https://en.wikipedia.org/wiki/Prisoner%27s_dilemma) (PD) is a classical game analyzed in game theory, which is widely used to (attempt to) model social/economical interaction. It's a "dilemma" as, if exploited to explain the emergence of altruism in human or in general animal society, it fails badly at a first glance.

The classical situation-representation of the PD is that of two prisoners whose conviction depends on their mutual cooperation. It is easier understood though if illustrated in terms of a trade-off game (closed bag exachange):

*Two people meet and exchange closed bags, with the understanding that one of them contains money, and the other contains a purchase. Either player can choose to honor the deal by putting into his or her bag what he or she agreed, or he or she can defect by handing over an empty bag.*

It is obvious that for both players the winning strategy is to NOT cooperate.

Things changes when the interaction between the two individuals is iterated, in that case a more altruist attitude (strategy) is expected to emerge. The goal of this project is to test this hypothesis.

Mathematically the PD can be expressed with very basic linear algebra. The key component is the **Payoff matrix** $M$, which quantify the reward each player gets depending on whether she cooperated or not (defect):

$$
M = 
\begin{pmatrix} 
R & S \\
T & P 
\end{pmatrix}
$$

with $T,R,S,P$ integers that satisfy the following conditions:

$$
T>R>P>S; \quad 2R > T+S
$$

for example $T=3$, $R=2$, $P=1$ and $S=0$, or  $T=5$, $R=3$, $P=2$, $S=0$. Each player choice (move) can be represented by one of the two axis in ${\rm I\!R}^2$, i.e. $u_C=\begin{pmatrix} 1 \\ 0 \end{pmatrix}$ or $u_D=\begin{pmatrix} 0 \\ 1 \end{pmatrix}$, where the first coordinate stands for *Cooperate* and the second for *Defect*. Being $u_1$ and $u_2$ their rewards $r_1$ and $r_2$ can be computed then as:

$$
r_1 = u_1^T M u_2
\quad
\quad
r_2 = u_2^T M u_1
$$

In an Iterative Prisoner's Dilemma (IPD), two players play prisoner's dilemma more than once in succession and they remember previous actions of their opponent and change their strategy accordingly. The winning strategy is the one which yields to a larger reward at the end of the IPD.

The strategy can be represented as a function which outputs either $u_C$ or $u_D$. Such function can depend on the opponent's history of moves, her on history of moves, on the number of moves played till that moment and so on, but it can only be based on a probability density function. Possible strategies are:

* **Nice guy**: always cooperate (the function's output is always $u_D$)
* **Bad guy**: always defect 
* **Mainly nice**: randomly defect $k\%$ of the times and cooperate $100-k\%$, $k<50$
* **Mainly bad**: randomly defect $k\%$ of the times and cooperate $100-k\%$, $k>50$
* **tit-for-tat**: start by cooperating, then repeat what the opponent has done in the previous move 

Many more and much more complex strategies can be implemented. The strategy can even change during the IPD.


### Assignments

* Implement a simple IPD between two players implementing two given strategies. Study the evolution along the tournament confronting different strategies; study the overall outcome in the different configurations. 
* Implement a multiple players IPD (MPIPD) where several strategies play against each other in a roud-robin scheme
* Iterate what done in the previous task (repeated MPIPD, rMPIPD)  by increasing the population implementing a given strategy depending on the results that strategy achieved in the previous iteration
* (*difficult*) Implement a rMPIPD where strategies are allowed to mutate. The goal is to simulate the effect of genetic mutations and the effect of natura selection. A parameter (gene) should encode the attidue of an individual to cooperate, such gene can mutate randomly and the corresponding phenotype should compete in the MPIPD such that the best-fitted is determined.  


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import itertools


In [ ]:
from abc import ABC, abstractmethod

class Player(ABC):
    """Abstract class, declare the move method"""
    @abstractmethod
    def move(self):
        pass

In [ ]:
class NiceGuy(Player):
    """
    Player that always outputs the cooperate move
    """
    def __init__(self, r_history=[], move_history=[]):
        self.r_history = r_history
        self.move_history = move_history  
        
    def append_reward(self, reward):
        self.r_history.append(reward)
        
    def move(self):
        uc = [1,0]
        self.move_history.append(uc)
        return uc

    
class BadGuy(Player):
    """
    Player that always outputs the defect move
    """
    def __init__(self, r_history=[], move_history=[]):
        self.r_history = r_history
        self.move_history = move_history
        
    def append_reward(self, reward):
        self.r_history.append(reward)
        
    def move(self):
        ud = [0,1]
        self.move_history.append(ud)
        return ud

    
class KBadGuy(Player):
    """
    Player that outputs the defect move 
    with probability k/100
    """
    def __init__(self, k, r_history=[], move_history=[]):
        self.k = k
        self.r_history = r_history
        self.move_history = move_history
        
    def append_reward(self, reward):
        self.r_history.append(reward)
        
    def move(self):
        uc = [1,0]
        ud = [0,1]
        if np.random.rand() > 1-(self.k/100):
            u = ud
        else:
            u = uc        
        self.move_history.append(u)
        return u
 
    
class Tit4Tat(Player):
    """
    Player, the first move is cooperate, 
    the subsequent moves are the same as the last 
    element in self.input_history
    """
    def __init__(self, r_history=[], move_history=[], input_history=[]):
        self.r_history = r_history
        self.move_history = move_history 
        self.input_history = input_history

    def append_reward(self, reward):
        self.r_history.append(reward)
        
    def append_input(self, inp):
        self.input_history.append(inp)
        
    def move(self):
        if len(self.input_history) == 0:
            u = [1,0]
        else:
            u = self.input_history[-1]
        self.move_history.append(u)
        return u
    
    
class Tit4TatMP(Player):
    """
    Player that starts with cooperate and then cooperates only if a fraction of players
    above or equal c_treshold has cooperated.
    """
    def __init__(self, r_history=[], move_history=[], input_history=[], c_threshold=0.5):
        self.r_history = r_history
        self.move_history = move_history 
        self.input_history = input_history
        self.c_threshold = c_threshold
    
    def append_reward(self, reward):
        self.r_history.append(reward)
        
    def append_input(self, m_inp):
        """
        multiplayer input m_inp should be an array or list with N rows for N players,
        in every row the list [1,0] or [0,1], it also works with [1] or [0].
        The number of input players can change. 
        """
        self.input_history.append(m_inp)
        
    def move(self):
        if len(self.input_history) == 0:
            u = [1,0]
        else:
            last_inp = np.array(self.input_history[-1])[:,0]
            if np.sum(last_inp)/np.shape(last_inp)[0] >= self.c_threshold:
                u = [1,0]
            else:
                u = [0,1]
        self.move_history.append(u)
        return u
        
        
class GrimTriggerMP(Player):
    """
    Player that cooperates until a numbers of players >= d_threshold defect in a turn,
    from now on he defects in every turn. 
    Setting d_threshold=0 means that it needs only one defecting player
    to start defecting itself.
    """
    def __init__(self, r_history=[], move_history=[], input_history=[], d_threshold=0.5):
        self.r_history = r_history
        self.move_history = move_history 
        self.input_history = input_history
        self.d_threshold = d_threshold
        self.defected = False
    
    def append_reward(self, reward):
        self.r_history.append(reward)  
        
    def append_input(self, m_inp):
        """
        multiplayer input m_inp should be an array or list with N rows for N players,
        in every row the list [1,0] or [0,1], it also works with [1] or [0].
        The number of input players can change. 
        """
        self.input_history.append(m_inp)
    
    def move(self):  
        if not self.defected and len(self.input_history) > 0:
            # check if it has been defected in the last turn
            last_inp = last_inp = np.array(self.input_history[-1])[:,0]
            if np.sum(last_inp)/np.shape(last_inp)[0] < 1 - self.d_threshold:
                self.defected = True
#                 print("debug DEFECTED")
                
        if not self.defected:
            u = [1,0]
        else:
            u = [0,1]
        self.move_history.append(u)
        return u

    
    
class LearningPlayer(Player):
    """
    Player that use past rewards in order to decide the next move
    """
    def __init__(self, r_history=[], move_history=[]):
        self.r_history = r_history
        self.move_history = move_history 
    
    def append_reward(self, reward):
        self.r_history.append(reward)
    

    def move(self):
        # maybe is better to use numpy
        
        # random move if its the first move
        if len(self.r_history) == 0:
            if np.random.rand() >= 0.5:
                u = [1,0]
            else:
                u = [0,1]
        else:
            move_cat_history = [move[0] for move in self.move_history]
            move_cat_history = [-1 if move == 0 else 1 for move in move_cat_history]
            r_cat_history = []
            for i in range(len(self.r_history)):
                r_cat_history.append(self.r_history[i] * move_cat_history[i])
            if sum(r_cat_history) > 0:
                # cooperate if rewards>0
                u = [1,0]
            elif sum(r_cat_history) < 0:
                # defect if rewards<0
                u = [0,1]
            else:
                # random choice if parity
                if np.random.rand() >= 0.5:
                    u = [1,0]
                else:
                    u = [0,1]
                
        self.move_history.append(u)
        return u

## Examples of players classes

In [ ]:
nice_guy = NiceGuy()

for i in range(10):
    u = nice_guy.move()
print(u)
print(nice_guy.move_history)
print(nice_guy.r_history)


In [ ]:
pino = KBadGuy(30)
for i in range(10):
    u = pino.move()
print(u)
print(pino.move_history)

In [ ]:

t4t_guy = Tit4Tat()
u = t4t_guy.move()
print("first move ",u)

print(t4t_guy.move_history)
t4t_guy.input_history.append([0,1])
u = t4t_guy.move()
print("second move (mirrored) ", u)

In [ ]:
t4t_m = Tit4TatMP([],[],[],0.5)

u = t4t_m.move()
print("first move ", u)

m_inp = [[1,0], [0,1], [0,1]]

t4t_m.append_input(m_inp)

u = t4t_m.move()
print("second move ", u)

m_inp = [[1], [0], [0], [1], [1]]
t4t_m.append_input(m_inp)

u = t4t_m.move()
print("third move ", u)

In [ ]:
# with d_threshold=0 it needs one defecting player to change
grim_trigger = GrimTriggerMP([],[],[],0)
u = grim_trigger.move()
print("first move is cooperate ", u)

# one defector over 4 other players
m_inp = [[1,0], [1,0], [1,0], [0,1]]
grim_trigger.append_input(m_inp)

u = grim_trigger.move()
print("second move ", u)

m_inp = [[1], [1], [1], [1]]
grim_trigger.append_input(m_inp)

u = grim_trigger.move()
print("third move ", u)

u = grim_trigger.move()
print("fourth move ", u)

# Part 1: 2 players IPD
Implement a simple IPD between two players implementing two given strategies. Study the evolution along the tournament confronting different strategies; study the overall outcome in the different configurations.

## KBadGuy vs Tit4Tat

In [ ]:
# Payoff matrix
M = np.array([[2,0],[3,1]])

# for practicity we are using u as horizontal arrays
# initialize players instances
p1 = Tit4Tat([],[],[])
p2 = KBadGuy(40,[],[])

# game loop
for i in range(100):
    u1 = p1.move()
    u2 = p2.move()
    #append p2 move, necessary for the tit4tat
    p1.append_input(u2)
    
    p1.append_reward(np.array(u1) @ M @ np.array(u2).T)
    p2.append_reward(np.array(u2) @ M @ np.array(u1).T)

# visualization
fig, ax = plt.subplots(1,2, figsize = (15,6))
ax[0].plot(p1.r_history, label=("p1"))
ax[0].plot(p2.r_history, label=("p2"))
ax[0].legend()
ax[0].set_xlabel("moves")
ax[0].set_ylabel("reward")
ax[0].set_title("Reward for every move")

ax[1].plot(np.cumsum(p1.r_history), label=("p1"))
ax[1].plot(np.cumsum(p2.r_history), label=("p2"))
ax[1].legend()
ax[1].set_xlabel("moves")
ax[1].set_ylabel("cumulative reward")
ax[1].set_title("Cumulative reward")
ax[1].grid(linestyle=":")


print("player1 final reward ", sum(p1.r_history))
print("player2 final reward ", sum(p2.r_history))

## Tit4Tat vs Tit4Tat

In [ ]:
# Payoff matrix
M = np.array([[2,0],[3,1]])

# for practicity we are using u as horizontal arrays
# initialize players instances
p1 = Tit4Tat([],[],[])
p2 = Tit4Tat([],[],[])

# game loop
for i in range(100):
    u1 = p1.move()
    p2.append_input(u1)
    
    u2 = p2.move()
    p1.append_input(u2)
        
    p1.append_reward(np.array(u1) @ M @ np.array(u2).T)
    p2.append_reward(np.array(u2) @ M @ np.array(u1).T)
    
# visualization    
fig, ax = plt.subplots(1,2, figsize = (15,6))
ax[0].plot(p1.r_history, label=("p1"))
ax[0].plot(p2.r_history, label=("p1"))
ax[0].set_ylim(0,3)
ax[0].legend()

ax[1].plot(np.cumsum(p1.r_history), label=("p1"))
ax[1].plot(np.cumsum(p2.r_history), label=("p2"))
ax[1].legend()


print("player1 final reward ", sum(p1.r_history))
print("player2 final reward ", sum(p2.r_history))

print("As expected two Tit4Tat players cooperates and obtain reward 2*i")

## KBadGuy vs KBadGuy

In [ ]:
# Payoff matrix
M = np.array([[2,0],[3,1]])

# initialize players instances
p1 = KBadGuy(10,[],[])
p2 = KBadGuy(30,[],[])

# game loop
for i in range(100):
    u1 = p1.move()
#     p2.append_input(u1)
    
    u2 = p2.move()
#     p1.append_input(u2)
        
    p1.append_reward(np.array(u1) @ M @ np.array(u2).T)
    p2.append_reward(np.array(u2) @ M @ np.array(u1).T)
    
# visualization
fig, ax = plt.subplots(1,2, figsize = (15,6))
ax[0].plot(p1.r_history, label=("p1"))
ax[0].plot(p2.r_history, label=("p2"))
ax[0].legend()
ax[0].set_xlabel("moves")
ax[0].set_ylabel("reward")
ax[0].set_title("Reward for every move")

ax[1].plot(np.cumsum(p1.r_history), label=("p1"))
ax[1].plot(np.cumsum(p2.r_history), label=("p2"))
ax[1].legend()
ax[1].set_xlabel("moves")
ax[1].set_ylabel("cumulative reward")
ax[1].set_title("Cumulative reward")
ax[1].grid(linestyle=":")


print("player1 final reward ", sum(p1.r_history))
print("player2 final reward ", sum(p2.r_history))

print("If two KBadGuy are playing, the one with bigger k tends to win")

## Explore the entire range of KBadGuys

Analize the connection between the two *k* values and the total reward value
running multiple games with different *k*

In [ ]:

# Payoff matrix
M = np.array([[2,0],[3,1]])

k_p1 = np.arange(0, 101, 10)
k_schedule = np.zeros((121,2))

for i in range(0,121,11):
    k_schedule[i:i+11, 0] = k_p1[i//11]
    k_schedule[i:i+11, 1] = k_p1[:]


reward_history = []
for n in range(np.shape(k_schedule)[0]):
    reward_game = [0,0]
    p1 = KBadGuy(k_schedule[n,0],[],[])
    p2 = KBadGuy(k_schedule[n,1],[],[])
    for i in range(100):              
        u1 = p1.move()
        u2 = p2.move()

        reward1 = np.array(u1) @ M @ np.array(u2).T
        reward2 = np.array(u2) @ M @ np.array(u1).T
        p1.append_reward(reward1)
        p2.append_reward(reward2)
        
        reward_game[0] += reward1
        reward_game[1] += reward2
        
    reward_history.append(reward_game)
    
reward_history = np.asarray(reward_history)
reward_2d = np.reshape(reward_history, (11,11,2))

# visualization    
fig, ax = plt.subplots(1,2, figsize = (15,6))
ax[0].plot(reward_history[:,0], label=("p1"))
ax[0].plot(reward_history[:,1], label=("p2"))
ax[0].legend()
ax[0].grid(linestyle=":")
ax[0].set_title("final rewards for the combinations of k")
ax[1].plot(np.sum(reward_history,1), label=("p1"))
ax[1].grid(linestyle=":")
ax[1].set_title("Sum of the final reward for both players")


fig, ax = plt.subplots(1,3, figsize = (22,6))
im0 = ax[0].imshow(reward_2d[:,:,0], origin='lower')
ax[0].set_xticks(range(11))
ax[0].set_yticks(range(11))
ax[0].set_xticklabels(range(0,110,10))
ax[0].set_yticklabels(range(0,110,10))
ax[0].set_xlabel("p1 k")
ax[0].set_ylabel("p2 k")
ax[0].set_title("P1 rewards depending on the combination of k")
plt.colorbar(im0, ax=ax[0])

im1 = ax[1].imshow(reward_2d[:,:,1], origin='lower')
ax[1].set_xticks(range(11))
ax[1].set_yticks(range(11))
ax[1].set_xticklabels(range(0,110,10))
ax[1].set_yticklabels(range(0,110,10))
ax[1].set_xlabel("p1 k")
ax[1].set_ylabel("p2 k")
ax[1].set_title("P2 rewards depending on the combination of k")
plt.colorbar(im1, ax=ax[1])

im2 = ax[2].imshow(np.sum(reward_2d, 2), origin='lower')
ax[2].set_xticks(range(11))
ax[2].set_yticks(range(11))
ax[2].set_xticklabels(range(0,110,10))
ax[2].set_yticklabels(range(0,110,10))
ax[2].set_xlabel("p1 k")
ax[2].set_ylabel("p2 k")
ax[2].set_title("Sum of P1, P2 rewards")
plt.colorbar(im2, ax=ax[2])

## Introducing the Learning Player

Experiment with learning players

In [ ]:
# Payoff matrix
M = np.array([[2,0],[3,1]])

p1 = KBadGuy(30,[],[])
p2 = LearningPlayer([],[])

for i in range(100):
    u1 = p1.move()
    u2 = p2.move()
    
    p1.append_reward(np.array(u1) @ M @ np.array(u2).T)
    p2.append_reward(np.array(u2) @ M @ np.array(u1).T)
    
fig, ax = plt.subplots(1,2, figsize = (15,6))
ax[0].plot(p1.r_history, label='KBadGuy')
ax[0].plot(p2.r_history, label='LearningPlayer')
ax[0].set_xlabel("moves")
ax[0].set_ylabel("reward")
ax[0].set_title("Reward for every move")
ax[0].legend()
ax[1].plot(np.cumsum(p1.r_history), label='KBadGuy')
ax[1].plot(np.cumsum(p2.r_history), label='LearningPlayer')
ax[1].grid(linestyle=":")
ax[1].set_xlabel("moves")
ax[1].set_ylabel("cumulative reward")
ax[1].set_title("Cumulative reward")
ax[1].legend()


print("KBadGuy final reward ", sum(p1.r_history))
print("LearningPlayer final reward ", sum(p2.r_history))

### Learning Player vs Learning Player

In [ ]:
# Payoff matrix
M = np.array([[2,0],[3,1]])

p1 = LearningPlayer([],[])
p2 = LearningPlayer([],[])

for i in range(100):
    u1 = p1.move()
    u2 = p2.move()
    
    p1.append_reward(np.array(u1) @ M @ np.array(u2).T)
    p2.append_reward(np.array(u2) @ M @ np.array(u1).T)

    
fig, ax = plt.subplots(1,2, figsize = (15,6))
ax[0].plot(p1.r_history, label='LearningPlayer1')
ax[0].plot(p2.r_history, label='LearningPlayer2')
ax[0].set_xlabel("moves")
ax[0].set_ylabel("reward")
ax[0].set_title("Reward for every move")
ax[0].legend()
ax[1].plot(np.cumsum(p1.r_history), label='LearningPlayer1')
ax[1].plot(np.cumsum(p2.r_history), label='LearningPlayer2')
ax[1].grid(linestyle=":")
ax[1].set_xlabel("moves")
ax[1].set_ylabel("cumulative reward")
ax[1].set_title("Cumulative reward")
ax[1].legend()


print("KBadGuy final reward ", sum(p1.r_history))
print("LearningPlayer final reward ", sum(p2.r_history))

### Learning player against every k bad player

In [ ]:
# Payoff matrix
M = np.array([[2,0],[3,1]])

k_p1 = np.arange(0, 101, 10)
k_schedule = np.arange(101)
# k_schedule = np.flip(k_schedule)

reward_history = []
for n in range(np.shape(k_schedule)[0]):
    reward_game = [0,0]
    p1 = KBadGuy(k_schedule[n],[],[])
    p2 = LearningPlayer([],[])
   
    for i in range(100):          
        u1 = p1.move()
        u2 = p2.move()

        reward1 = np.array(u1) @ M @ np.array(u2).T
        reward2 = np.array(u2) @ M @ np.array(u1).T
        p1.append_reward(reward1)
        p2.append_reward(reward2)
        
        reward_game[0] += reward1
        reward_game[1] += reward2
        
    reward_history.append(reward_game)
    
reward_history = np.asarray(reward_history)
# reward_2d = np.reshape(reward_history, (10,10,2))

# print(reward_history)
# visualization    
fig, ax = plt.subplots(1,2, figsize = (15,6))
ax[0].plot(reward_history[:,0], label=("KBadGuy"))
ax[0].plot(reward_history[:,1], label=("LearningPlayer"))
ax[0].legend()
ax[0].grid(linestyle=":")
ax[0].set_title("final rewards for the combinations of k")
ax[1].plot(np.sum(reward_history,1), label=("p1"))
ax[1].grid(linestyle=":")
ax[1].set_title("Sum of the final reward for both players")

### Learning player against every k bad player 
multiple run in order to reduce the randomness effects

In [ ]:
# Payoff matrix
M = np.array([[2,0],[3,1]])

k_p1 = np.arange(0, 101, 10)
k_schedule = np.arange(101)


reward_history = []
games_per_batch = 100
for n in range(np.shape(k_schedule)[0]):
    reward_game_batch = [0,0]
    for j in range(games_per_batch):
        reward_game = [0,0]
        p1 = KBadGuy(k_schedule[n],[],[])
        p2 = LearningPlayer([],[])
        for i in range(100):      
            u1 = p1.move()
            u2 = p2.move()

            reward1 = np.array(u1) @ M @ np.array(u2).T
            reward2 = np.array(u2) @ M @ np.array(u1).T
            p1.append_reward(reward1)
            p2.append_reward(reward2)

            reward_game[0] += reward1
            reward_game[1] += reward2
            
        reward_game_batch[0] += reward_game[0]
        reward_game_batch[1] += reward_game[1]

    reward_history.append(reward_game_batch)
    
reward_history = np.asarray(reward_history)/games_per_batch



# visualization    
fig, ax = plt.subplots(1,2, figsize = (15,6))
ax[0].plot(reward_history[:,0], label=("KBadGuy"))
ax[0].plot(reward_history[:,1], label=("LearningPlayer"))
ax[0].legend()
ax[0].grid(linestyle=":")
ax[0].set_title("final rewards for the combinations of k")
ax[1].plot(np.sum(reward_history,1))
ax[1].grid(linestyle=":")
ax[1].set_title("Sum of the final reward for both players")

print("we can see that a LearningPlayer obtain a higher or similar final reward playing every KBadGuy")
print("So it wins against a NiceGuy, a MainlyNiceGuy, and has similar results to a MainlyBadGuy and a BadGuy")
print("notice that the results are very noisy")

# Part 2: multiplayer IPD


In order to extend our game to a N-Players scenario, we started from the normal form matrix proposed in [1]:
<br><br>

$$
M = 
\begin{pmatrix} 
R & K & S \\
T & L & P 
\end{pmatrix}
$$
<br>
where the first collumn identifies the case in which all the N-1 players decide to collaborate (strategy $(C...C)$), the second one the case in which at least one player decides to defect (strategy $(C...D)$), and the last one represent the case where every N-1 player chooses to defect (strategy $(D...D)$).
<br><br>
Furthermore, the strategy D has to continue to be the dominant one, thus we added few more constraint on the values of $R,K,S,T,L,P$:

$$
\begin{cases}
T > R > L > K > P > S\\
2R > T+S\\
2K > S+L
\end{cases}
$$
<br>
These constraints lead to the following payoff matrix that will be used for the rest of the project:
<br><br>
$$
M = 
\begin{pmatrix} 
7 & 3 & 0 \\
9 & 5 & 1 
\end{pmatrix}
$$
<br>
Thanks to the linear algebra formulation of the problem, we won't need to define the players payoffs for every combination in the outcome set $\{C,D\}^N$.<br>
In fact, the player $i$ payoff can be easly computed as:
$$
r_i = u_i^T M u_{N-1}
$$
<br>
where $u_{N-1}$ is defined as:

$$
u_{N-1} =
\begin{cases}
[1,0,0]\quad \textrm{if all the N-1 players play C}\\
[0,1,0]\quad \textrm{if at least one of the N-1 players plays D}\\
[0,0,1]\quad \textrm{if all the N-1 players play D}
\end{cases}
$$
<br>
When we refer to an IPD, new strategies can be defined such as:
* **tit-for-tat-MPIPD**: start by cooperating, then repeat what the majority of the opponets has done in the previous move.
* **Grim-Trigger guy**: start cooperating and keep cooperating till the other does too. If the other defects, start defecting from now on, no matter what the other does.



From game theory we know that the last round will be played selfishly and a Nash Equilibrium will be played from rational players; thus the last outcome will be $D...D$ (every player chooses to defect), leading to a low payoff equal to 1.
However it is evident that, if the number of game iterations $T$ is greater than $T_{th} = \frac{14}{6}$, cooperating till the last but one round would be the best strategy given that:

$$
(T-1)*(C...C)+(D...D) > (C...D) + (T-1)(D...D)\\
7(T-1)+1 > 9 + (T-1)\\
\textrm{that with:}\quad T=3>T_{th}\quad \textrm{leads to:}\quad 15 > 11
$$





References:<br>
[1]: https://www.researchgate.net/publication/295855229_Iterated_symmetric_three-player_prisoner%27s_dilemma_game

In [ ]:
T=9; R=7; L=5; K=3; P=1; S=0;

# Payoff matrix
M2 = np.array([[R, K, S],[T, L, P]])
print("M2")
print(M2)



#P_list = player list
#GI = number of Game Iterations
def MPIPD(P_list, GI):
    # game loop
    for i in range(GI):

        U = np.array([P_list[i].move() for i in range(len(P_list))]).T #matrix with all the players move of this round

        for k in range(len(P_list)):   #for each player   
            #get moves of others player (if CC, CD or DD)
            #problems with case k=0 and k=len(P) solved in silly way
            if(k!=0 and k!= len(P_list)-1): 
                others_moves = np.prod(U[:,0:k], axis=1) * np.prod(U[:, k:], axis=1)
            elif(k==(len(P_list)-1)):
                others_moves = np.prod(U[:,0:k], axis=1)
            else:
                others_moves = np.prod(U[:,1:], axis=1)


            if(np.array(others_moves == [1,0]).all()): #case CC

                others_moves = np.array([1,0,0])

            elif(np.array(others_moves == [0,0]).all()): #case CD

                others_moves = np.array([0,1,0])

            else: #case DD

                others_moves = np.array([0,0,1])


            #check if player is tit4tat, if yes, update input history
            if(isinstance(P_list[k], Tit4Tat)):

                if(others_moves[0] == 1): #the others played CC
                    P_list[k].append_input([1,0])

                else: #the others played CD or DD                
                    P_list[k].append_input([0,1])

            if(isinstance(P_list[k], Tit4TatMP) or isinstance(P_list[k], GrimTriggerMP)):
                P_list[k].append_input(np.delete(U, k, axis=1)) #copy all moves but the player i-th one
                
                

            P_list[k].append_reward(U[:,k] @ M2 @ others_moves.T)

   

            
#P_list = player list
#P_plot_index = indexes of the player that we want to plot
def plot_rewards_MPIPD(P_list, P_plot_index=None):        

    if(P_plot_index == None): 
        P_plot_index = range(len(P_list))
        
        
    fig, ax = plt.subplots(1,2, figsize = (15,6))

    for i in P_plot_index:

        print("player"+ str(i) +" final reward "+ str(sum(P_list[i].r_history)) +". Player type: " + str(type(P_list[i])))
        ax[0].plot(P_list[i].r_history)
        ax[1].plot(np.cumsum(P_list[i].r_history), label=str(type(P_list[i])))
        ax[1].legend()


In [ ]:
# initialize players instances

p1 = Tit4Tat([],[],[])
p2 = KBadGuy(40,[],[])
p3 = NiceGuy([],[])
p4 = BadGuy([],[])
p5 = KBadGuy(60,[],[])

P = [p1, p2, p3, p4, p5]

#run the game
MPIPD(P, 100)

#plot the results
plot_rewards_MPIPD(P)

plt.show()



p1 = Tit4TatMP([],[],[],0.6)
p2 = KBadGuy(60,[],[])
p3 = NiceGuy([],[])
p4 = GrimTriggerMP([],[],[],0.4)
p5 = LearningPlayer([],[])
p6 = KBadGuy(30,[],[])
p7 = LearningPlayer([],[])

P = [p1, p2, p3, p4, p5, p6, p7]

MPIPD(P, 50)
plot_rewards_MPIPD(P, [0,3,4,5])

plt.show()

# Part 3: reapeted multiplayer IPD with population increasing
Iterate what done in the previous task (repeated MPIPD, rMPIPD)  by increasing the population implementing a given strategy depending on the results that strategy achieved in the previous iteration

# Part4: rMPIPD with mutating strategies
(*difficult*) Implement a rMPIPD where strategies are allowed to mutate. The goal is to simulate the effect of genetic mutations and the effect of natura selection. A parameter (gene) should encode the attidue of an individual to cooperate, such gene can mutate randomly and the corresponding phenotype should compete in the MPIPD such that the best-fitted is determined. 